In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd


# Import Data Downloaded from Google BigQuery

In [3]:
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/data_from_bigquery.csv')
print(df.columns)
print(df.shape)
df[df.id == 55661532]

Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags',
       'view_count', 'id_1', 'title_1', 'body_1', 'accepted_answer_id_1',
       'answer_count_1', 'comment_count_1', 'community_owned_date_1',
       'creation_date_1', 'favorite_count_1', 'last_activity_date_1',
       'last_edit_date_1', 'last_editor_display_name_1',
       'last_editor_user_id_1', 'owner_display_name_1', 'owner_user_id_1',
       'parent_id_1', 'post_type_id_1', 'score_1', 'tags_1', 'view_count_1'],
      dtype='object')
(60570, 40)


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count,id_1,title_1,body_1,accepted_answer_id_1,answer_count_1,comment_count_1,community_owned_date_1,creation_date_1,favorite_count_1,last_activity_date_1,last_edit_date_1,last_editor_display_name_1,last_editor_user_id_1,owner_display_name_1,owner_user_id_1,parent_id_1,post_type_id_1,score_1,tags_1,view_count_1
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,55661552,2,3,NaN,2019-04-13 02:45:30.12 UTC,NaN,2019-04-13 03:18:14.417 UTC,2019-04-13 03:18:14.417 UTC,NaN,1157106.0,NaN,7693832.0,NaN,1,2,python,1816,55661630,NaN,<p>The answer to your new question is that you...,NaN,NaN,0,NaN,2019-04-13 03:09:34.107 UTC,NaN,2019-04-13 03:09:34.107 UTC,NaN,NaN,NaN,NaN,7631480.0,55661532,2,0,NaN,NaN
1398,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,55661552,2,3,NaN,2019-04-13 02:45:30.12 UTC,NaN,2019-04-13 03:18:14.417 UTC,2019-04-13 03:18:14.417 UTC,NaN,1157106.0,NaN,7693832.0,NaN,1,2,python,1816,55661552,NaN,<p>You're running your program as a shell scri...,NaN,NaN,3,NaN,2019-04-13 02:50:19.66 UTC,NaN,2019-04-13 02:50:19.66 UTC,NaN,NaN,NaN,NaN,116908.0,55661532,2,1,NaN,NaN


In [4]:
#Select only relevant columns and rename
df2 = df[['id', 'title', 'body', 'accepted_answer_id', 'answer_count', 'view_count', 'id_1', 'body_1', 'score_1']]
df2.columns = ['question_id', 'question_title_raw', 'question_body_raw', 'accepted_answer_id', 'answer_count', 'view_count', 'answer_id', 'answer_body_raw', 'answer_score']
df2.head()

,question_id,question_title_raw,question_body_raw,accepted_answer_id,answer_count,view_count,answer_id,answer_body_raw,answer_score
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,55661552,2,1816,55661630,<p>The answer to your new question is that you...,0
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,55661660,2,2521,55661660,<p>You can't use <code>socket.socket(socket.AF...,0
2,55661929,How to display 5 numbers per line from a list?,<p>How to display 5 numbers per line from a li...,55661981,4,125,55662134,<p>There can be different approaches to the pr...,0
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,55662821,2,24,55662821,<p>Define list as <code>[]</code> and use <cod...,0
4,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,55662821,2,24,55662828,<p>I modified your code slightly:</p>\n\n<pre>...,0


# Preprocessing


## Add accepted_answers column (1's and 0's) 

In [5]:
def is_accepted_answer(row):
  """Returns whether the answer is the accepted answer or not"""
  return 1 if row['accepted_answer_id'] == row['answer_id'] else 0

In [6]:
df2['is_accepted_answer'] = df2.apply(lambda row: is_accepted_answer(row), axis=1)
df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,question_id,question_title_raw,question_body_raw,accepted_answer_id,answer_count,view_count,answer_id,answer_body_raw,answer_score,is_accepted_answer
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,55661552,2,1816,55661630,<p>The answer to your new question is that you...,0,0
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,55661660,2,2521,55661660,<p>You can't use <code>socket.socket(socket.AF...,0,1
2,55661929,How to display 5 numbers per line from a list?,<p>How to display 5 numbers per line from a li...,55661981,4,125,55662134,<p>There can be different approaches to the pr...,0,0
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,55662821,2,24,55662821,<p>Define list as <code>[]</code> and use <cod...,0,1
4,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,55662821,2,24,55662828,<p>I modified your code slightly:</p>\n\n<pre>...,0,0


 ## Drop unique questions with more than 10 answers  

In [7]:
print(df2.shape)

#Create dataframe of question_id and count of answer per question_id
answer_count = df2[['question_id','answer_id']].groupby(by="question_id").count()   

#Isolate dataframe to only question_ids with more than X answers
X=10
answer_count_Xplus = answer_count[answer_count.answer_id > X]

#Make an array of question_ids with more than X answers
unique_question_ids_Xplus = np.array(answer_count_Xplus.index)                         

#Isolate raw_data dataframe with question_ids with more than 10 answers each
raw_data_Xplus = df2[df2['question_id'].isin(unique_question_ids_Xplus)]     

#Drop questions from raw_data dataframe with more than 10 answers
df2 = df2.drop(raw_data_Xplus.index)                                          
print(df2.shape)

(60570, 10)
(60336, 10)


In [8]:
answer_count2 = df2[['question_id','answer_id']].groupby(by="question_id").count()
answer_count2.sort_values(by=['answer_id'], ascending=False).head(10)

,answer_id
question_id,
37084246,10
51456809,10
42658784,10
39440723,10
56156938,10
57018192,10
39042775,9
39080180,9
56853540,9


## Parse Question and Answer Body to Extract Content From Tags & Parse Text / Code
1) Concatenate Question Title and Body (Raw data concatenated) 

2) Run ExtractContent, ParseText, and ParseCode (on each raw column). New Columns include:  

-- question_title_raw.............(no extractcontent, extractcode, nor extracttext <- b/c question-title does not have html tags or code)

-- question_body_raw  
-- question_body.................(extractContent)  
-- question_body_text.................(extractText)    
-- question_body_code.................(extractCode)      

-- question_title_body_raw  
-- question_title_body.................(extractContent)  
-- question_title_body_text.................(extractText)      
-- question_title_body_code.................(extractCode)        

-- answer_body_raw  
-- answer_body.................(extractContent)     
-- answer_text.................(extractText)      
-- answer_code.................(extractCode)        


In [9]:
# Import parse.py file
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/master/helper_functions')
import parse

In [10]:
df3=df2
print(df3['question_title_raw'].isnull().values.any())
print(df3['question_body_raw'].isnull().values.any())

False
False


In [11]:
# 1. Concatenate question title and body
df3["question_title_body_raw"] = df3['question_title_raw'] + df3['question_body_raw']
df3.head(5)

,question_id,question_title_raw,question_body_raw,accepted_answer_id,answer_count,view_count,answer_id,answer_body_raw,answer_score,is_accepted_answer,question_title_body_raw
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,55661552,2,1816,55661630,<p>The answer to your new question is that you...,0,0,"When I run my socket program, there get: `impo..."
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,55661660,2,2521,55661660,<p>You can't use <code>socket.socket(socket.AF...,0,1,"with socket.socket(socket.AF_INET, socket.SOCK..."
2,55661929,How to display 5 numbers per line from a list?,<p>How to display 5 numbers per line from a li...,55661981,4,125,55662134,<p>There can be different approaches to the pr...,0,0,How to display 5 numbers per line from a list?...
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,55662821,2,24,55662821,<p>Define list as <code>[]</code> and use <cod...,0,1,create list with spectial type - Spyder error ...
4,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,55662821,2,24,55662828,<p>I modified your code slightly:</p>\n\n<pre>...,0,0,create list with spectial type - Spyder error ...


In [12]:
# 2. Parse Text and Parse Code

import time
start = time.time()

df3.loc[:,'question_body'] = df3['question_body_raw'].apply(lambda row: parse.extractContent(row), 1)
df3.loc[:,'question_body_text'] = df3['question_body_raw'].apply(lambda x: parse.extractText(x))
df3.loc[:,'question_body_code'] = df3['question_body_raw'].apply(lambda x: parse.extractCode(x))

df3.loc[:,'question_title_body'] = df3['question_title_body_raw'].apply(lambda row: parse.extractContent(row), 1)
df3.loc[:,'question_title_body_text'] = df3['question_title_body_raw'].apply(lambda x: parse.extractText(x))
df3.loc[:,'question_title_body_code'] = df3['question_title_body_raw'].apply(lambda x: parse.extractCode(x))

df3.loc[:,'answer_body'] = df3['answer_body_raw'].apply(lambda row: parse.extractContent(row), 1)
df3.loc[:,'answer_text'] = df3['answer_body_raw'].apply(lambda x: parse.extractText(x))
df3.loc[:,'answer_code'] = df3['answer_body_raw'].apply(lambda x: parse.extractCode(x))
df3.head(5)

print("Time: %0.2f seconds" % (time.time()-start))


Time: 237.74 seconds


In [13]:
df3.columns

Index(['question_id', 'question_title_raw', 'question_body_raw',
       'accepted_answer_id', 'answer_count', 'view_count', 'answer_id',
       'answer_body_raw', 'answer_score', 'is_accepted_answer',
       'question_title_body_raw', 'question_body', 'question_body_text',
       'question_body_code', 'question_title_body', 'question_title_body_text',
       'question_title_body_code', 'answer_body', 'answer_text',
       'answer_code'],
      dtype='object')

In [14]:
df4 = df3[['question_id', 
           'question_title_raw',
           'question_body_raw', 'question_body', 'question_body_text','question_body_code',
           'question_title_body_raw', 'question_title_body', 'question_title_body_text', 'question_title_body_code',
           'answer_id',
           'answer_body_raw','answer_body','answer_text','answer_code',
           'answer_count', 'view_count', 'answer_score', 'is_accepted_answer']]
df4.head(2)

,question_id,question_title_raw,question_body_raw,question_body,question_body_text,question_body_code,question_title_body_raw,question_title_body,question_title_body_text,question_title_body_code,answer_id,answer_body_raw,answer_body,answer_text,answer_code,answer_count,view_count,answer_score,is_accepted_answer
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,I write a socket server programmer:\n#-*- codi...,I write a socket server programmer:\n\nwhen I ...,#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,"When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,55661630,<p>The answer to your new question is that you...,The answer to your new question is that you ca...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,1816,0,0
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,I have a socket server program:\n#!/usr/bin/en...,I have a socket server program:\n\nwhen I run ...,#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,"with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,55661660,<p>You can't use <code>socket.socket(socket.AF...,"You can't use socket.socket(socket.AF_INET, so...",You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,2521,0,1


# Create Columns for Text/Code and Code/Text Ordering

In [15]:
df4['question_title_body_order_textcode'] = df4['question_title_body_text'] + df4['question_title_body_code']
df4['question_title_body_order_codetext'] = df4['question_title_body_code'] + df4['question_title_body_text']
df4['answer_body_order_textcode'] = df4['answer_text'] + df4['answer_code']
df4['answer_body_order_codetext'] = df4['answer_code'] + df4['answer_text']
df4.head(5)

,question_id,question_title_raw,question_body_raw,question_body,question_body_text,question_body_code,question_title_body_raw,question_title_body,question_title_body_text,question_title_body_code,answer_id,answer_body_raw,answer_body,answer_text,answer_code,answer_count,view_count,answer_score,is_accepted_answer,question_title_body_order_textcode,question_title_body_order_codetext,answer_body_order_textcode,answer_body_order_codetext
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,I write a socket server programmer:\n#-*- codi...,I write a socket server programmer:\n\nwhen I ...,#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,"When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,55661630,<p>The answer to your new question is that you...,The answer to your new question is that you ca...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,1816,0,0,"When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE..."
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,I have a socket server program:\n#!/usr/bin/en...,I have a socket server program:\n\nwhen I run ...,#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,"with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,55661660,<p>You can't use <code>socket.socket(socket.AF...,"You can't use socket.socket(socket.AF_INET, so...",You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,2521,0,1,"with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE..."
2,55661929,How to display 5 numbers per line from a list?,<p>How to display 5 numbers per line from a li...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",55662134,<p>There can be different approaches to the pr...,There can be different approaches to the probl...,There can be different approaches to the probl...,x=[] #your given list\nfor t in range(len(x))...,4,125,0,0,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",There can be different approaches to the probl...,x=[] #your given list\nfor t in range(len(x))...
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,55662821,<p>Define list as <code>[]</code> and use <cod...,Define list as [] and use append to add elemen...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',...",2,24,0,1,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',..."
4,55662778,create

In [16]:
idx=5
print(df4['question_title_body_text'][idx])
print("################################")
print(df4['question_title_body_code'][idx])
print("################################")
print(df4['question_title_body_order_codetext'][idx])

print("################################")

print(df4['answer_text'][idx])
print("################################")
print(df4['answer_code'][idx])
print("################################")
print(df4['answer_body_order_codetext'][idx])


Python - Inserting and Updating python dict simultaneouslyso I have a list of dicts that looks like this: 

And I want an output that looks like this: 

That is, I want every  to be the key and have it append the  and  if it belongs to the same field. 
Note: I want to do this WITHOUT using a for loop (apart from the loop to iterate through the list). I want to use python dict functions like  and  etc.
Any optimized solutions would be really helpful.
################################
[{
  'field': {
    'data': 'F1'
  },
  'value': F1Value1,
  'date': datetime.datetime(2019, 3, 1, 0, 0)
}, {
  'field': {
    'data': 'F2'
  },
  'value': F2Value1,
  'date': datetime.datetime(2019, 2, 5, 0, 0)
}, {
  'field': {
    'data': 'F2'
  },
  'value': F2Value2,
  'date': datetime.datetime(2019, 2, 7, 0, 0)
}]
[
  {
    'F1': [
      {
        'value': F1Value1,
        'date': datetime.datetime(2019, 3, 1, 0, 0)
      }
    ]
  },
  {
    'F2': [
      {
        'value': F2Value1,
        'date': 

## Fill nan with empty string

In [17]:
df4 = df4.fillna("")
df4.head(5)

,question_id,question_title_raw,question_body_raw,question_body,question_body_text,question_body_code,question_title_body_raw,question_title_body,question_title_body_text,question_title_body_code,answer_id,answer_body_raw,answer_body,answer_text,answer_code,answer_count,view_count,answer_score,is_accepted_answer,question_title_body_order_textcode,question_title_body_order_codetext,answer_body_order_textcode,answer_body_order_codetext
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,I write a socket server programmer:\n#-*- codi...,I write a socket server programmer:\n\nwhen I ...,#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,"When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,55661630,<p>The answer to your new question is that you...,The answer to your new question is that you ca...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,1816,0,0,"When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE..."
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,I have a socket server program:\n#!/usr/bin/en...,I have a socket server program:\n\nwhen I run ...,#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,"with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,55661660,<p>You can't use <code>socket.socket(socket.AF...,"You can't use socket.socket(socket.AF_INET, so...",You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,2521,0,1,"with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE..."
2,55661929,How to display 5 numbers per line from a list?,<p>How to display 5 numbers per line from a li...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",55662134,<p>There can be different approaches to the pr...,There can be different approaches to the probl...,There can be different approaches to the probl...,x=[] #your given list\nfor t in range(len(x))...,4,125,0,0,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",There can be different approaches to the probl...,x=[] #your given list\nfor t in range(len(x))...
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,55662821,<p>Define list as <code>[]</code> and use <cod...,Define list as [] and use append to add elemen...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',...",2,24,0,1,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',..."
4,55662778,create

In [18]:
df4.isna().any()

question_id                           False
question_title_raw                    False
question_body_raw                     False
question_body                         False
question_body_text                    False
question_body_code                    False
question_title_body_raw               False
question_title_body                   False
question_title_body_text              False
question_title_body_code              False
answer_id                             False
answer_body_raw                       False
answer_body                           False
answer_text                           False
answer_code                           False
answer_count                          False
view_count                            False
answer_score                          False
is_accepted_answer                    False
question_title_body_order_textcode    False
question_title_body_order_codetext    False
answer_body_order_textcode            False
answer_body_order_codetext      

# Save to Google Drive

In [19]:
df4.to_csv('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/raw_data_preprocessed.csv', index=False)

# Dataframe Columns


In [20]:
# question_id

# question_title_raw  # no extractcontent, extractcode, nor extracttext <- b/c question-title does not have html tags or code

# question_body_raw
# question_body
# question_body_text
# question_body_code

# question_title_body_raw
# question_title_body
# question_title_body_text
# question_title_body_code

# answer_id

# answer_body_raw
# answer_body
# answer_text
# answer_code

# answer_count
# view_count
# answer_score
# is_accepted_answer